# Summary baseline

## Load the data

In [1]:
root_dir = '../../'
src_dir = 'src'
data_dir = 'data/corpus'
models_dir = 'data/models'

In [2]:
import os
import sys

In [3]:
sys.path.append(os.path.join(root_dir, src_dir))

In [4]:
version = 'v2'
corpus_filename = f'wikidata_corpus_{version}.json'

In [5]:
from training import TrainingCorpus

In [6]:
corpus = TrainingCorpus()
corpus.load(os.path.join(root_dir, data_dir, corpus_filename))

In [7]:
corpus.size

3294

---

## Builds pseudo-docs

In [8]:
from collections import defaultdict

In [9]:
pseudodocs_dict = defaultdict(str)

In [10]:
for doc_id in corpus.docs:
    text = corpus.get_text(doc_id)
    label = corpus.target[doc_id][0]
    pseudodocs_dict[label] += ' ' + text

## Summarize pseudo-docs

In [11]:
entities = list(pseudodocs_dict.keys())
num_entities = len(entities)
pseudodocs = [pseudodocs_dict[e_id] for e_id in entities]

In [12]:
summaries = []

In [13]:
from transformers import pipeline

/home/nvidia/anaconda3/envs/iml/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10000). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [14]:
model_name = 'facebook/bart-large-cnn'

In [15]:
summarizer = pipeline('summarization', model=model_name)

In [16]:
min_len = 10
max_len = 50
do_sample = False

In [17]:
for i in range(num_entities):
    doc = pseudodocs[i]
    summary_text = summarizer(doc, min_length=min_len, max_length=min(max_len, len(doc.split())), do_sample=do_sample)[0]['summary_text']
    summaries.append(summary_text)

IndexError: index out of range in self

In [ ]:
from pprint import pprint

In [ ]:
pprint(summaries)

---

## Compute tf-idf on each pseudo-document

### Define corpus

In [ ]:
clean_pseudodocs = []

for i in range(num_entities):
    doc = pseudodocs[i].lower()
    clean_doc = ' '.join(TrainingCorpus.tokenize(doc))
    clean_pseudodocs.append(clean_doc)

len(clean_pseudodocs)

# Fit tf-idf vectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer = TfidfVectorizer(lowercase=False, tokenizer=lambda x: x.split())
vectorizer.fit(clean_pseudodocs)

## Save tf-idf weights into a dict

In [ ]:
vectorized_docs = vectorizer.transform(clean_pseudodocs)
vectorized_docs.shape

In [ ]:
tfidf_weights = []

for i in range(num_entities):
    doc_vector = vectorized_docs[i].toarray().reshape(-1)
    weights = {}
    
    for j, w in enumerate(doc_vector):
        feature_name = vectorizer.get_feature_names()[j]
        if w > 0:
            weights[feature_name] = w
    
    tfidf_weights.append(weights)

## Compute baseline summaries

In [ ]:
min_len = 1
max_len = 30

In [ ]:
def compute_score(idx, text):
    if text:
        weights = tfidf_weights[idx]
        tokenized_text = TrainingCorpus.tokenize(text.lower())
        score = 0

        if tokenized_text:
            for token in tokenized_text:
                if token in weights:
                    score += weights[token]
        text_len = len(text.split())
        score = score/text_len
        return score
    else:
        return 0

In [ ]:
import nltk

In [ ]:
final_summaries = []

for i in range(num_entities):
    # split on sentences
    final_summary = []
    sentences = nltk.tokenize.sent_tokenize(summaries[i])
    prev_score = 0
    
    for sentence in sentences:
        print(sentence)
        score = compute_score(i, sentence)
        print(score)
        delta_score = score - prev_score
        
        if delta_score > 0:
            final_summary.append(sentence)
        else:
            break
        
        prev_score = score
    
    final_summaries.append('.'.join(final_summary))

In [ ]:
for s in final_summaries:
    print(s)
    print()

---